In [213]:
import pandas as pd

df = pd.read_excel('output.xlsx', index_col=0)

df.head()

,Adm No,Stdnt Name,Form,Stream,Exam,Subject,Term,Yearr,Marks,examID,subjectCode
13935,7779,Peter Mugwe Waweru,4,S,END TERM,Business studies,1,2019,67,21,565
13937,7779,Peter Mugwe Waweru,4,S,END TERM,Business studies,2,2019,73,24,565
13940,7779,Peter Mugwe Waweru,4,S,END TERM,Chemistry,1,2019,54,21,233
13942,7779,Peter Mugwe Waweru,4,S,END TERM,Chemistry,2,2019,49,24,233
13950,7779,Peter Mugwe Waweru,4,S,END TERM,CRE,1,2019,35,21,313


In [214]:
# list unique exams
df.examID.unique()

array([21, 24, 22, 25, 26, 23, 27])

In [215]:
# filter to remain only with examID 21, 22, 23 for training and testing. 
# we shall try and predict outcome wuth other exams as out of sample data
df_filtered = df[df['examID'] < 24]
df_filtered.head()

,Adm No,Stdnt Name,Form,Stream,Exam,Subject,Term,Yearr,Marks,examID,subjectCode
13935,7779,Peter Mugwe Waweru,4,S,END TERM,Business studies,1,2019,67,21,565
13940,7779,Peter Mugwe Waweru,4,S,END TERM,Chemistry,1,2019,54,21,233
13950,7779,Peter Mugwe Waweru,4,S,END TERM,CRE,1,2019,35,21,313
13960,7779,Peter Mugwe Waweru,4,S,END TERM,English,1,2019,46,21,101
13975,7779,Peter Mugwe Waweru,4,S,END TERM,History $ Gov,1,2019,50,21,311


In [216]:
# filter out unwanted columns
needed_cols = ['Adm No', 'Form', 'Stream', 'Marks', 'examID', 'subjectCode']
df_filtered = df_filtered[needed_cols]
df_filtered.head()

,Adm No,Form,Stream,Marks,examID,subjectCode
13935,7779,4,S,67,21,565
13940,7779,4,S,54,21,233
13950,7779,4,S,35,21,313
13960,7779,4,S,46,21,101
13975,7779,4,S,50,21,311


In [217]:
# rename the Adm No column to eliinate the space
df_filtered.rename(columns={'Adm No': 'admNo'}, inplace=True)
df_filtered.head()

,admNo,Form,Stream,Marks,examID,subjectCode
13935,7779,4,S,67,21,565
13940,7779,4,S,54,21,233
13950,7779,4,S,35,21,313
13960,7779,4,S,46,21,101
13975,7779,4,S,50,21,311


In [218]:
# since we are considering sciences, maths and languages, we remove other subjects
df_filtered = df_filtered[df_filtered['subjectCode'] < 299]
df_filtered.head()

,admNo,Form,Stream,Marks,examID,subjectCode
13940,7779,4,S,54,21,233
13960,7779,4,S,46,21,101
13985,7779,4,S,52,21,102
13990,7779,4,S,66,21,121
14000,7779,4,S,49,21,232


In [219]:
# certain students may be missing a mark in a given science or language or maths,
# we need to remove them as we will be considering all 3 sciences and 2 languages and maths
# since we have dropped all subjects that are not sciences, math or languages,
# each student needs to have 6 subject marks for each exam

# first we create 3 data frames for each exam
df_21 = df_filtered[df_filtered.examID == 21]
df_22 = df_filtered[df_filtered.examID == 22]
df_23 = df_filtered[df_filtered.examID == 23]

# secondly we remove duplicate subjects from each of the above dataframes
df_21.groupby(['subjectCode', 'admNo'])
df_22.groupby(['subjectCode', 'admNo'])
df_23.groupby(['subjectCode', 'admNo'])

# we could also do as below but it takes much more execution time
# df_21.drop_duplicates(subset=['subjectCode', 'admNo']) 

# third, we merge the 3 dataframes
df_merged = pd.merge(df_21, df_22, how='outer')
df_merged = pd.merge(df_merged, df_23, how='outer')

# finally remove all students whose records appear less than 18 times ( 3 exams * 6 subjects)
df_filtered = df_merged[df_merged.groupby('admNo')['admNo'].transform('size') == 18]
df_filtered

,admNo,Form,Stream,Marks,examID,subjectCode
1209,8178,2,H,36,21,231
1210,8178,2,H,19,21,233
1211,8178,2,H,40,21,101
1212,8178,2,H,32,21,102
1213,8178,2,H,13,21,121
1214,8178,2,H,23,21,232
1215,8179,2,S,35,21,231
1216,8179,2,S,20,21,233
1217,8179,2,S,35,21,101
1218,8179,2,S,39,21,102


In [220]:
# now we need to populate the average columns
# we need to convert all sciences to have same code, similar to languages and we will have a code for maths
# we will use 1 for maths, 2 for sciences and 3 for languages

df_filtered.loc[df_filtered.subjectCode == 121, 'subjectCode'] = 1
df_filtered.loc[df_filtered.subjectCode.isin([231, 232, 233]), 'subjectCode'] = 2
df_filtered.loc[df_filtered.subjectCode.isin([101, 102]), 'subjectCode'] = 3
df_filtered

,admNo,Form,Stream,Marks,examID,subjectCode
1209,8178,2,H,36,21,2
1210,8178,2,H,19,21,2
1211,8178,2,H,40,21,3
1212,8178,2,H,32,21,3
1213,8178,2,H,13,21,1
1214,8178,2,H,23,21,2
1215,8179,2,S,35,21,2
1216,8179,2,S,20,21,2
1217,8179,2,S,35,21,3
1218,8179,2,S,39,21,3


In [267]:
# now we group the subjects for a particular student and find the averages
x = df_filtered.groupby(['admNo', 'subjectCode'], as_index=False).mean()

# x_columns = ['admNo', 'subjectCode', 'Form', 'Marks', 'examID']
# x = x.set_index(x_columns)
# x['maths'] = 0
x
y = x.pivot(index='admNo', columns='subjectCode', values='Marks')

z = pd.merge(x, y, how='outer')
# for student in x.admNo.unique():



MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False